In [2]:
reset -fs

In [148]:
import numpy as np
import math

In [567]:
class Space():
    def __init__(self, space_type):
        self.space_type = space_type
        
class Prop(Space):
    def __init__(self, purchase_price):
        super().__init__('property')
        self.for_sale = True
        self.mortgaged = False
        self.houses = None
        self.owner = None
        self.purchase_price = purchase_price
        
    def mortgage(self):
        self.mortgaged = True
        self.for_sale = True
        self.purchase_price = round(self.purchase_price * 0.55)
        
        return round(self.purchase_price / 1.1)
        
    def unmortgage(self):
        self.for_sale = False
        self.mortgaged = False
        self.purchase_price = round(self.purchase_price / 0.55)
        
        return round(self.purchase_price * 0.55)

class Color(Prop):
    def __init__(self, purchase_price, static_rents, group):
        super().__init__(purchase_price)
        self.static_rents = static_rents
        self.group = group
        self.house_price = (math.floor(group / 2) + 1) * 50
        self.houses = 0
        self.for_sale = True
        self.mortgaged = False
        
    def rent(self, group_ownership, roll):
        if self.houses == 0:
            return math.floor(group_ownership + 1) * self.static_rents[0]
        else:
            return self.static_rents[self.houses]
        
    def buy_houses(self, n):
        self.houses += n
        return n * self.house_price
    
    def sell_houses(self, n):
        self.houses -= n
        return self.house_price / 2

class Railroad(Prop):
    def __init__(self):
        super().__init__(200)
        self.purchase_price = 200
        self.group = 8
        self.for_sale = True
        self.mortgaged = False
        
    def rent(self, group_ownership, roll):
        return 25 * (2**((group_ownership * 4) - 1))
    
class Utility(Prop):
    def __init__(self):
        super().__init__(150)
        self.purchase_price = 150
        self.group = 9
        self.for_sale = True
        self.mortgaged = False
        
    def rent(self, group_ownership, roll):
        return roll * (5**(group_ownership * 2))
    
class Taxes(Space):
    def __init__(self, amount):
        super().__init__('taxes')
        self.amount = amount
        
class Free_Parking(Space):
    def __init__(self):
        super().__init__('free parking')
        self.prize = 100
        
class Card(Space):
    def __init__(self, space_type):
        super().__init__(space_type)
        self.order = self.shuffle()
        
    def shuffle(self):
        deck = np.arange(16)
        np.random.shuffle(deck)
        
        return deck
    
    def draw_card(self):
        card = self.order[-1]
        self.order = self.order[:-1]
        if len(self.order) == 0:
            self.order = self.shuffle()
        
        return card

In [804]:
class SuperMonopoly():
    def __init__(self, n_players, n_boards):
        self.n_boards = n_boards
        self.n_players = n_players
        self.board = self.construct_board()
        self.group_locations = [[1, 3], [6, 8, 9], [11, 13, 14],
                                [16, 18, 19], [21, 23, 24], [26, 27, 29],
                                [31, 32, 34], [37, 39], [5, 15, 25, 35, ],
                                [12, 28]]
        self.hotels = np.array([32, 12] * n_boards).reshape(n_boards, 2)
        self.positions = np.zeros((n_players, 2), dtype=int)
        self.in_jail = np.zeros((n_players,), dtype=int)
        self.act_player = self.first_player()
        self.deeds = np.zeros((n_boards, n_players, 10))
        self.equity = np.zeros((n_boards, n_players, 10))
        self.money = np.full((n_players, ), 1500 + (500 * (n_boards - 1)))
        self.doubles_counter = 0
        self.winner = None
        self.n_moves = 0

    def dice(self):
        roll = np.random.randint(1, 7, size=2)
        return roll[0], roll[1]

    def first_player(self):
        rolls = [sum(dice()) for p in range(self.n_players)]
        highest_roll = max(rolls)
        rolls = [p if p == highest_roll else 0 for p in rolls]

        while(highest_roll != sum(rolls)):
            rolls = [sum(dice()) if p != 0 else 0 for p in rolls]
            highest_roll = max(rolls)
            rolls = [p if p == highest_roll else 0 for p in rolls]

        return np.argmax(np.array(rolls))

    def purchase_property(self, spot):
        prop = self.board[spot]
        group_size = len(self.group_locations[prop.group])
        self.deeds[spot[0], self.act_player,
                   prop.group] += (12 / group_size) / 17
        self.money[self.act_player] -= prop.purchase_price
        prop.for_sale = False
        prop.unmortgage()
        prop.owner = self.act_player

        if self.deeds[spot[0], self.act_player, prop.group] > 11 / 17:
            self.equity[spot[0], self.act_player, prop.group] = 1

    def mortgage_property(self, spot):
        prop = self.board[spot]
        group_size = len(self.group_locations[prop.group])
        self.deeds[spot[0], self.act_player,
                   prop.group] -= (12 / group_size) / 17
        self.money[self.act_player] += prop.mortgage()
        prop.owner = self.act_player
        self.equity[spot[0], :, prop.group] = 0

    def unmortgage_property(self, spot):
        prop = self.board[spot]
        group_size = len(self.group_locations[prop.group])
        self.deeds[spot[0], self.act_player,
                   prop.group] += (12 / group_size) / 17
        self.money[self.act_player] -= prop.unmortgage()
        prop.owner = self.act_player

        if self.deeds[spot[0], self.act_player, prop.group] > 11 / 17:
            self.equity[spot[0], self.act_player, prop.group] = 1

    def purchase_equity(self, board_number, group, equity, price):
        owner = np.argmax(self.deeds[board_number, :, group])
        self.equity[board_number, self.act_player, group] += equity
        self.equity[board_number, owner, group] -= equity
        self.money[self.act_player] -= price
        self.money[owner] += price

    def purchase_houses(self, board_number, group, n=1):
        self.deeds[board_number, self.act_player, group] += n * (1 / 17)

        for prop_location in self.group_locations[group]:
            prop = self.board[board_number][prop_location]
            if n + prop.houses > 4:
                self.houses[board_number][0] += prop.houses
                self.houses[board_number][1] -= 1
            else:
                self.houses[board_number][0] -= n

            self.money[self.act_player] -= prop.buy_houses(n)

    def sell_houses(self, board_number, group, n):
        self.deeds[board_number, self.act_player, group] -= n * (1 / 17)

        for prop_location in self.group_locations[group]:
            prop = self.board[board_number][prop_location]
            if prop.houses > 4:
                self.houses[board_number][0] -= prop.houses - n
                self.houses[board_number][1] += 1
            else:
                self.houses[board_number][0] += n

            self.money[self.act_player] += prop.sell_houses(n)

    def pay_rent(self, spot, dice_vals):
        prop = self.board[spot]
        ownership = self.deeds[spot[0], prop.owner, prop.group]
        r = prop.rent((ownership * 17) / 12, sum(dice_vals))

        if r >= self.money[self.act_player]:
            r = self.money[self.act_player]

        self.money[self.act_player] -= r

        if ownership > 12 / 17:
            self.money[prop.owner] += r
        else:
            self.money += self.equity[spot[0], :, prop.group] * r

    def pay_out_of_jail(self):
        self.board[self.positions[self.act_player][0], 20].prize += 50
        self.money[self.act_player] -= 50
        self.in_jail[self.act_player] = 0

    def roll_dice(self, agent):
        dice_vals = self.dice()
        doubles = dice_vals[0] == dice_vals[1]
        spot = (self.positions[self.act_player][0],
                self.positions[self.act_player][1] + sum(dice_vals))

        jail_time = self.in_jail[self.act_player]

        if ((jail_time > 0) and (jail_time <= 3)):
            if doubles:
                self.in_jail[self.act_player] = 0
                self.land_on_spot(spot, dice_vals)
            else:
                self.pay_out_of_jail()
                self.end_turn()
        elif doubles and self.doubles_counter == 3:
            self.doubles_counter = 0
            self.go_to_jail()
        else:
            if spot[1] > 39:
                spot = ((spot[0] + 1) % self.n_boards, spot[1] % 39)
                self.money[self.act_player] += 200

            self.land_on_spot(spot, dice_vals)

            if doubles:
                self.doubles_counter += 1
            else:
                self.doubles_counter = 0
                self.end_turn()

    def end_turn(self):
        self.n_moves += 1
        self.doubles_counter = 0
        if len(self.money[self.money > 0]) <= 1:
            self.winner = np.argmax(self.money)
        else:
            self.act_player = (self.act_player + 1) % self.n_players

            while self.money[self.act_player] <= 0:
                self.act_player = (self.act_player + 1) % self.n_players

    def available_actions(self):

        properties_owned = self.deeds[:, self.act_player, :8].flatten()
        not_properties_owned = np.hstack((self.deeds[:, :self.act_player - 1, :8],
                                          self.deeds[:, self.act_player:, :8])).flatten()
        
        for i in range(len(properties_owned)):
            if properties_owned[i] > (11 / 17):
                b = i // 40
                b_i = i % 40
                if (properties_owned[i] == (16 / 17)) and (self.hotels[b][1] >= 3):
                    actions.append['build']
                    break
                elif self.hotels[b][0] >= 3:
                    actions.append['build']
                    break

        if (len(properties_owned[properties_owned > 11 / 17]) > 0):
            actions.append('invest')
            
        if np.sum(properties_owned) > 0:
            actions.append('liquidate')

    def land_on_spot(self, spot, dice_vals, card_multiplier=1):
        self.positions[self.act_player] = spot
        space_type = self.board[spot].space_type

        if space_type == 'property':
            if self.board[spot].for_sale:
                agent.purchase()
            else:
                self.pay_rent(spot, dice_vals)

        if space_type == 'taxes':
            self.money[self.act_player] -= self.board[spot].amount
            self.board[spot[0], 20].prize += self.board[spot].amount
        if space_type == 'free parking':
            self.money[self.act_player] += self.board[spot[0], 20].prize
            self.board[spot[0], 20].prize = 100
        if space_type == 'chance':
            self.chance(spot, self.board[spot].draw_card(), dice_vals)
        if space_type == 'community chest':
            self.community_chest(spot, self.board[spot].draw_card(), dice_vals)
        if space_type == 'go to jail':
            self.go_to_jail()

    def chance(self, spot, card_number, dice_vals):
        if card_number == 0:
            self.land_on_spot((spot[0], 39), dice_vals)
        if (card_number == 1) or (card_number == 2):
            self.money[self.act_player] += 50
        if card_number == 3:
            if spot[1] > 5:
                self.money[self.act_player] += 200 * self.n_boards
            self.land_on_spot((spot[0], 5), dice_vals)
        if card_number == 4:
            self.money[self.act_player] -= 15
            self.board[spot[0], 20].prize += 15
        if (card_number == 5) or (card_number == 11):
            if spot[1] > 35:
                self.money[self.act_player] += 200 * self.n_boards
                self.land_on_spot((spot[0], 5), dice_vals, 2)
            else:
                r = int(5 + (10 * (round(spot[1] / 10))))
                self.land_on_spot((spot[0], r), dice_vals, 2)
        if card_number == 6:
            self.go_to_jail()
        if card_number == 7:
            self.land_on_spot((spot[0], spot[1] - 3), dice_vals)
        if card_number == 8:
            give_money = np.full((self.n_players,), 50)
            give_money[self.act_player] = -50 * (self.n_players - 1)
            self.money += give_money
        if card_number == 9:
            owned = self.deeds[spot[0], self.act_player]
            owned = owned[owned > 0]
            house_amount = np.sum(owned[owned < 5]) * 25
            hotel_amount = (np.sum(owned[owned > 4]) / 5) * 100
            self.money[self.act_player] -= house_amount + hotel_amount
            self.board[spot[0], 20].prize += house_amount + hotel_amount
        if card_number == 10:
            self.money[self.act_player] += 200
            self.land_on_spot(((spot[0] + 1) % self.n_boards, 0), dice_vals)
        if card_number == 12:
            if spot[1] > 11:
                self.money[self.act_player] += 200 * self.n_boards
            self.land_on_spot((spot[0], 11), dice_vals)
        if card_number == 13:
            if spot[0] > 12:
                self.land_on_spot((spot[0], 12), dice_vals, 2)
            elif spot[0] > 28:
                self.land_on_spot((spot[0], 28), 2)
            else:
                self.money[self.act_player] += 200 * self.n_boards
                self.land_on_spot((spot[0], 12), dice_vals, 2)
        if card_number == 14:
            if spot[1] > 24:
                self.money[self.act_player] += 200 * self.n_boards
            self.land_on_spot((spot[0], 24), dice_vals)
        if card_number == 15:
            self.money[self.act_player] += 150

    def community_chest(self, spot, card_number, dice_vals):
        if (card_number == 0) or (card_number == 3):
            self.money[self.act_player] -= 50
            self.board[spot[0], 20].prize += 50
        if card_number == 1:
            get_money = np.full((self.n_players,), -10)
            get_money[self.act_player] = 10 * (self.n_players - 1)
            self.money += get_money
        if (card_number == 2) or (card_number == 5) or (card_number == 13):
            self.money[self.act_player] += 50
        if card_number == 4:
            self.go_to_jail()
        if (card_number == 6) or (card_number == 7) or (card_number == 12):
            self.money[self.act_player] += 100
        if (card_number == 8) or (card_number == 9):
            self.money[self.act_player] += 25
        if card_number == 10:
            self.money[self.act_player] -= 100
            self.board[spot[0], 20].prize += 100
        if card_number == 11:
            self.money[self.act_player] += 20
        if card_number == 14:
            self.money[self.act_player] += 200
            self.land_on_spot(
                ((spot[0] + 1) % self.n_boards, 0), dice_vals, agent)
        if card_number == 15:
            owned = self.deeds[spot[0], self.act_player]
            owned = owned[owned > 0]
            house_amount = np.sum(owned[owned < 5]) * 40
            hotel_amount = (np.sum(owned[owned > 4]) / 5) * 115
            self.money[self.act_player] -= house_amount + hotel_amount
            self.board[spot[0], 20].prize += house_amount + hotel_amount

    def go_to_jail(self):
        self.in_jail[self.act_player] += 1
        self.positions[self.act_player][1] = 10
        self.end_turn()

    def construct_board(self):
        board = []
        for i in range(self.n_boards):
            b = [Space('go'),
                 Color(60, [2, 10, 30, 90, 160, 250], 0),
                 Card('community chest'),
                 Color(60, [4, 20, 60, 180, 320, 450], 0),
                 Taxes(200),
                 Railroad(),
                 Color(100, [6, 30, 90, 270, 400], 1),
                 Card('chance'),
                 Color(100, [6, 30, 90, 270, 400], 1),
                 Color(120, [8, 40, 100, 300, 450, 600], 1),
                 Space('jail'),
                 Color(140, [10, 50, 150, 450, 625, 750], 2),
                 Utility(),
                 Color(140, [10, 50, 150, 450, 625, 750], 2),
                 Color(160, [12, 60, 180, 500, 700, 900], 2),
                 Railroad(),
                 Color(180, [14, 70, 200, 550, 750, 950], 3),
                 None,
                 Color(180, [14, 70, 200, 550, 750, 950], 3),
                 Color(200, [16, 80, 220, 600, 800, 1000], 3),
                 None,
                 Color(220, [18, 90, 250, 700, 875, 1050], 4),
                 None,
                 Color(220, [18, 90, 250, 700, 875, 1050], 4),
                 Color(240, [20, 100, 300, 750, 925, 1100], 4),
                 Railroad(),
                 Color(260, [22, 110, 330, 800, 975, 1150], 5),
                 Color(260, [22, 110, 330, 800, 975, 1150], 5),
                 Utility(),
                 Color(280, [24, 120, 360, 850, 1025, 1200], 5),
                 Space('go to jail'),
                 Color(300, [26, 130, 390, 900, 1100, 1275], 6),
                 Color(300, [26, 130, 390, 900, 1100, 1275], 6),
                 None,
                 Color(320, [28, 150, 450, 1000, 1200, 1400], 6),
                 Railroad(),
                 None,
                 Color(350, [35, 175, 500, 1100, 1300, 1500], 7),
                 Taxes(75),
                 Color(400, [50, 200, 600, 1400, 1700, 2000], 7)]

            b[17] = b[2]
            b[33] = b[2]
            b[22] = b[7]
            b[36] = b[7]
            b[20] = Free_Parking()
            board.append(b)

        return np.array(board)

liquidate - sell your least valuable asset 
build - buy houses
invest - offer deals 
trade - offer trades
roll

In [751]:
t = SuperMonopoly(2,3)

In [749]:
t.unmortgage_property((0,5))

In [837]:
class Agent():
    def __init__(self, game, epsilon=0.1, alpha=1.0):
        self.game = game
        self.epsilon = epsilon
        self.alpha = alpha
        self.Q = np.zeros((3, 3, game.n_boards * 40), dtype=int)
        self.current_state = (1, 1, 0)
    
    def offer_investment(self, board_number, property_group, equity=0.25):
        price = np.random.uniform() * (property_group / 28) * 2000
        self.game.purchase_equity(board_number, property_group, equity, price)
        
        return price

    def determine_state(self):

        act_money = self.game.money[self.game.act_player]
        average_money = np.sum(self.game.money) / self.game.n_players

        if (act_money / average_money) < 0.5:
            money_vector = 0
        elif (act_money / average_money) > 2:
            money_vector = 2
        else:
            money_vector = 1

        act_deeds = np.sum(self.game.deeds[:, self.game.act_player, :])
        average_deed = np.sum(self.game.deeds) / self.game.n_players

        if (act_deeds / average_deeds) < 0.5:
            property_vector = 0
        elif (act_deeds / average_deeds) > 2:
            property_vector = 2
        else:
            property_vector = 1

        position_vector = self.game.positions[self.game.act_player]
        position_vector = (40 * position_vector[0]) + position_vector[1]

        return (money_vector, property_vector, position_vector)

    def reward_func(self, money_change, asset_change):
        act_money = (self.game.money[self.game.act_player] + money_change)/ np.sum(self.game.money)

        deed_scale = np.array([1, 2, 3, 4, 5, 6, 7, 8, 5, 2]).reshape(1, 10)
        scaled_deeds = np.multiply(self.game.deeds, deed_scale)
        act_deeds = (np.sum(scaled_deeds[: ,self.game.act_player, :]) + asset_change) / np.sum(scaled_deeds)
        
        return (act_deeds / (1 + act_deeds)) + m

    def explore_actions(self):
        actions = self.game.available_actions()
        rewards = []
        
        if action == 'liquidate':
            # Liquidate cheapest asset
        if action == 'invest':
            # Invest randomly 
        if action == 'build':
            # Build the best asset you can afford
    
    def take_action(self, action):
        
        if action == 'liquidate':
            # Liquidate cheapest asset
        if action == 'invest':
            # Invest randomly 
        if action == 'build':
            # Build the best asset you can afford
        self.game.roll_dice()
            

IndentationError: expected an indented block (<ipython-input-837-732e57e54615>, line 47)